# Exécuter un script d’entraînement avec le SDK Python

Vous pouvez utiliser le SDK Python pour Azure Machine Learning afin de soumettre des scripts en tant que travaux. En utilisant des travaux, vous pouvez facilement suivre les paramètres d’entrée et les sorties lors de l’entraînement d’un modèle Machine Learning.

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Utiliser le SDK Python pour entraîner un modèle

Pour entraîner un modèle, vous allez d’abord créer le script **diabetes_training.py** dans le dossier **src**. Le script utilise le fichier **diabetes.csv** dans le même dossier que les données d’entraînement.

In [ ]:
Exécutez la cellule ci-dessous pour soumettre le travail qui entraîne un modèle de classification afin de prédire le diabète. 

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Use the Python SDK to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))


Run the cell below to submit the job that trains a classification model to predict diabetes. 

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-pythonv2-train",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)